| <div> <img src="https://storage.googleapis.com/open-ff-common/openFF_logo.png" width="100"/><br>Open-FF</div>|<h1>Exploring fracking chemicals<br>used near a selected location</h1>|<center><a href="https://www.fractracker.org/" title="FracTracker Alliance"><img src="https://storage.googleapis.com/open-ff-common/2021_FT_logo_icon.png" alt="FracTracker logo" width="100" height="100"><br>Sponsored by<br> FracTracker Alliance</a></center>|
|---|---|---|

#### This notebook is licensed under the MIT License. Please visit our [Github repo](https://github.com/gwallison/openFF) for more information.
#### For more information about this project, visit [our website](https://frackingchemicaldisclosure.wordpress.com/).
(Structure and instructions for this notebook are inspired by the [Environmental Enforcement Watch Project](https://environmentalenforcementwatch.org/data/notebooks) and their research notebooks. Some of their images may be used below.)

Note: This notebook pulls data from a recent copy of Open-FF's data set. This data sets is updated regularly, meaning that some of the results from your run may not exactly match those from a previous run.

## Summarizing fracking chemicals used near a selected location 
---
This is a Jupyter Notebook - a way to organize Python computer programming code. Hosting the notebook on Google Colab allows you to access and visualize data without actually needing to do any coding! Some code is left visible for individuals with a knowledge of Python or for those wondering how this site was put together. This allows for a more interactive user experience. 

This notebook will help you explore the chemicals that have been disclosed by the oil and gas industry to the FracFocus registry. FracFocus is an industry-created repository that references over 200,000 fracking events since 2011.  However, the FracFocus data presents many obstacles. Open-FF is an independent, open-source project to make the FracFocus data more robust and searchable and to provide some perspective about the chemicals reported. 

### How to use this notebook:
After you load some necessary code and data (Step 1, which can take a minute or more), you will indicate what location you want to explore (which state in Step 2, and the lat/lon in step 3). In Step 4 you will select the size of the area around your focal location to search for fracked wells. In Step 5, you will generate the summarized chemical list for all the wells in that area. And finally in Step 6 you can save a record of this analysis.

Before trying your own search, you can run the entire notebook with just the default values by just stepping through each cell to the end to see how it works.  To run a code cell (they have an empty bracket `[ ]` in the upper left corner), hover your mouse over that bracket.  It should display a "play" icon.  Clicking that icon will execute the code in that cell.

To perform more than one search, just start again at Step 2.  We only need to perform Step 1 once.

If you run into problems that you cannot remedy, please let me know! I'll try to get back to you as quick as possible.  (gary.allison@gmail.com)

## Step 1 -  Download the necessary code and data
Note that first time you run a step, Google will ask you if you really want to run this code.  Generally, you do not want to run code on Colab unless you trust the source.  In this case, you can just hit "Run Anyway".

This first step can take **a few minutes** while the Colab machines download data and code and prep the data sets.

In [ ]:
# # fetch the Open-FF code repository and master data files from remote storage
# # This can take two minutes or more

!git clone --depth 1  https://github.com/gwallison/openFF.git &>/dev/null;
!pip install itables  &>/dev/null;
!pip install geopandas  &>/dev/null;
!pip install reportlab &>/dev/null;
%run openFF/notebooks/Explore_near_location_support.py


## Step 2 - Enter your state of interest
In the text box above enter the state name.  (We use the state name to filter the data set to make it a reasonable size for processing)

**Then run the following cell:** 
It will make a data set specific to your state.  (It may take a minute or so.)

In [ ]:
df, lat_lon_input  = make_working_data_set(state_name_input)

## Step 3 - Enter the your location of interest

### Enter the latitude and longitude of you location of interest into the text box above.
A simple way to do that is
- find your location on a [Google map](https://www.google.com/maps/). 
    - Use the traditional map or satellite view to find it **OR**
    - If it is a specific well site, use the Browser's [Disclosure Index](https://storage.googleapis.com/open-ff-browser/Disclosure_Index.html) to find it by APINumber or WellName, click for the disclosure and then, click on the "Google Map" link for that well.
- Once you've found it on the Google map, right-click your mouse on your spot to show a menu of map options.
    -Select the option with the Latitude/Longitude coordinates.  That copies the values to the clipboard.
- Clear the input box above and paste the clipboard values into the box.

## Then run the next cell:
It will:
- use the lat/lon you entered as the focal location
- map that location as a check that it is what you want.

Notes about the map:

Run the cell below to show your selected location on a map and/or satellite image. Use the layers selector in the upper right corner to choose which is active.

Note that by clicking on the resulting map, the latitude and longitude of the point you click on is copied to the clipboard. If you want to adjust the focal point for subsequent analysis, just paste that lat/lon pair into the location selector above and re-run the following cell to make the clicked point the focal point.

In [ ]:
lat,lon = process_lat_lon_input(lat_lon_input)
display(maps.show_simple_map(lat,lon,clickable=True))

radius_input = show_radius_input()
display(radius_input)


## Step 4 - Choose size of focal circle around focal point
In the text box above, enter the radius around your focal point to search.  The radius is in FEET.  

Then run the following cell.   
It will:
- produce a list of the wells in that circle
- display a map of them
- graph the amount of water used in each disclosure

In [ ]:
radius_in_feet = process_radius_input(radius_input)
apis = get_apis(df,lat,lon,radius_in_feet)
print(f'Number of wells within radius: {len(apis)}')
t, dgb = show_well_info(apis,df)
radius_in_meters = radius_in_feet*0.3048
display(maps.showWells(df,lat,lon,apis,def_buffer=radius_in_meters))
show_water_used(dgb)

# Step 5 - What chemicals were used?
### All fracking jobs combined

Run the following cell to display a summary of chemicals reported in all fracking disclosures of this set. The legend for the resulting table is at the bottom of this notebook. 

Although proprietary records (if they are present) are summarized, other chemical records that cannot be resolved to valid CASRNs are excluded in the table below.  See individual disclosures to see all records.

In [ ]:
chem_obj = create_chem_summary(t)
show_chem_summary(chem_obj)
report_name_input = show_report_name_input()
display(md("### Enter text to use as report title"))
display(report_name_input)


# Step 6 - Record your results
At this point, you may want a record of the information you have compiled.
There are a handful of ways to do that:
- **Create a PDF report**.  Run the cell below and a PDF report will be created with the core information compiled in this notebook. Use the text box above to enter the name you want on the report. Then run the cell below. When it has completed, the will be a file named "report_test.pdf" in the file area of this colab page.
- **Save this Colab notebook**.  By saving this colab file on your computer or Google Drive, you can open it later and all of the output as you gernerated it should be there. To do this, Click on the "File" menu, click "Download" and select the "Download .ipynb" option.  Save the resulting file somewhere you will find it again.  To view it later, open [colab](https://colab.google/), and upload the file you've saved.
- **Save the downloaded data for these selected disclosures**. If you wish to download the full data for these disclosures, click on the "folder" icon in the left menu bar. Download the file named: `all_data_for_selected_wells.csv`

In [ ]:
save_pdf_report(name_input=report_name_input,well_list=dgb,c_obj=chem_obj,lat=lat,
                lon=lon,radius=radius_in_feet)

|Explanation of columns in the index|
| :---: |

| Column      | Description |
| :----: | :-------- |
|*composite_id*| at top is the CAS registration number of the chemical. **Click on this link** to display summary of this chemical for all of FracFocus (filtered data - up to last major update). Below are two names for the material: the upper is used by SciFinder as the primary name and the lower is the most common reported name in FracFocus| 
|*refs*|Click on the "csv" link to a csv-formatted file that has all the filtered records for this chemical.  Click on "PubChem" link for a PubChem web search for this chemical.  Click on "CompTox" for the summary page in that EPA resource|
|*total num records* |indicates the number of records across all wells summarized. A given chemical may appear more than once in a disclosure.|
|*num records with mass*|indicates the number of records with mass of this chemical in the **filtered** data. Without mass, a record only indicates **presence** in the disclosure.|
|*Total mass used (lbs)*| the sum of the computable mass of all records in the set|
|*Reportable quantity (lbs)*|indicates the quantity of the chemical (in pounds), above which uses must be reported if spilled, based on 40 CFR § 302.4 - Hazardous substances and reportable quantities. This is applicable throughout most industries however fracking is in general exempt from this requirement. We show it here to illustrate what those exemptions have given to the fracking industry. There are currently a total of about 100 chemicals on that list.|
|*Chemicals of Concern* |indicates if the chemical is on one of the following lists.  Type the name of the list into the Search box to limit to those chemicals.<br> - **CWA**: indicates that the chemical is on the [Clean Water Act list](https://comptox.epa.gov/dashboard/chemical_lists/CWA311HS) as compiled in EPA's CompTox<br>- **DWSHA**: indicated that the chemical is on the EPA's [Drinking Water Safety and Health Advisory](https://comptox.epa.gov/dashboard/chemical_lists/EPADWS) list<br>- **TEDX**: indicates that the chemical is on [The Endocrine Disruption Exchange](https://endocrinedisruption.org) list<br>- **prop_65**: indicates that the chemical is on [California's Proposition 65](https://oehha.ca.gov/proposition-65/proposition-65-list) list<br>- **EPA_PFAS**: indicates that the chemical is on EPA's comprehensive list of PFAS related compounds<br>- **UVCB**: indicates that the chemical is on the [TSCA list of "Unknown, Variable composition, Complex reaction products and Biological"](https://www.epa.gov/sites/default/files/2015-05/documents/uvcb.pdf) materials, meaning "these chemical substances cannot be represented by unique structures and molecular formulas."<br>- **diesel**: indicates that the chemical is [on the EPA list that](https://www.epa.gov/uic/diesel-fuels-hydraulic-fracturing-dfhf) "represent the most appropriate interpretation of the statutory term 'diesel fuels' to use for permitting diesel fuels hydraulic fracturing under the UIC Program nationwide.|
|*ChemInformatics*|This is a "hazard" summary based on the EPA's [ChemInformatics Dashboard](https://www.epa.gov/chemical-research/cheminformatics). For each chemical that the EPA has published summaries, Open-FF generates a single image that indicates level of hazard for EPA's twenty categories. See the table below or click on the specific chemical's report (its CAS number) for a different display of these results. For many chemicals reported by the EPA on that site, some of those hazard categories cannot be evaluated because the data are limited. See EPA's disclaimer for these data below or at their site.|